In [1]:
import utilities
import os
import socket
import tldextract
import pandas as pd
import csv

In [2]:
base_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Traffic"
dns = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/DNS/ip_domain.json")
categories_echo = utilities.get_directories_in_a_directory("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Traffic/Traffic_echo_0307_new")

#  Creat a dictionary for fqdns in dns

In [3]:
dict_fqdns = {}
for ip, fqdn in dns.items():
    if fqdn not in dict_fqdns:
        dict_fqdns[fqdn] = {'domain_name': '', 'SLD': '', 'subdomain' : '' ,'organization': '', 'ATSes': '', 'Party': ''}
        ext = tldextract.extract(fqdn)
        dict_fqdns[fqdn]['domain_name'] = ext.domain+"."+ext.suffix
        dict_fqdns[fqdn]['subdomain'] = ext.subdomain
        dict_fqdns[fqdn]['SLD'] = ext.domain
        dict_fqdns[fqdn]['organization'] = ""
        dict_fqdns[fqdn]['ATSes'] = ""
        dict_fqdns[fqdn]['Party'] = ""
dict_fqdns

{'mlis.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'mlis',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'd3p8zr0ffa9t17.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd3p8zr0ffa9t17',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'api.amazonalexa.com': {'domain_name': 'amazonalexa.com',
  'SLD': 'amazonalexa',
  'subdomain': 'api',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'device-metrics-us-2.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'device-metrics-us-2',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'd29r7idq0wxsiz.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd29r7idq0wxsiz',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'd37b6sgey2y8gt.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd37b6sgey2y8gt',
  'organization': '',
  'ATSes': '',
  'Part

In [4]:
len(dict_fqdns)

52

In [5]:
# add conflict ips
conflics = ['doctorpooch.com']

In [6]:
for fqdn in conflics:
    dict_fqdns[fqdn] = {'domain_name': '', 'SLD': '', 'subdomain' : '' ,'organization': '', 'ATSes': '', 'Party': ''}
    ext = tldextract.extract(fqdn)
    dict_fqdns[fqdn]['domain_name'] = ext.domain+"."+ext.suffix
    dict_fqdns[fqdn]['subdomain'] = ext.subdomain
    dict_fqdns[fqdn]['SLD'] = ext.domain
    dict_fqdns[fqdn]['organization'] = ""
    dict_fqdns[fqdn]['ATSes'] = ""
    dict_fqdns[fqdn]['Party'] = ""

In [7]:
len(dict_fqdns)

53

# Find Organizations

In [8]:
ddg_entities = utilities.get_files_in_a_directory("/home/c2/alexa/post-processing/tracker-radar/entities")

for fqdn, details in dict_fqdns.items():
    for entity in ddg_entities:
        entity_content = utilities.read_json(entity)
        if details['domain_name'] in entity_content["properties"]:
            dict_fqdns[fqdn]['organization'] = entity_content["name"]
            break

In [9]:
not_found_organizations = set()
for fqdn, details in dict_fqdns.items():
    if len(details['organization']) == 0:
        not_found_organizations.add(details['domain_name'])

In [10]:
not_found_organizations

{'acsechocaptiveportal.com',
 'amazon-dss.com',
 'amazonalexa.com',
 'amcs-tachyon.com',
 'chtbl.com',
 'dillilabs.com',
 'doctorpooch.com',
 'fireoscaptiveportal.com',
 'megaphone.fm',
 'mittendorf.net',
 'omny.fm',
 'podtrac.com',
 'voiceapps.com'}

### For organizations we didn't find on DDG, we use https://lookup.icann.org/lookup and crunchbase.com

In [11]:
manually_found_organs = {'acsechocaptiveportal.com' : 'Amazon Technologies, Inc.',
 'amazon-dss.com' : 'Amazon Technologies, Inc.',
 'amazonalexa.com': 'Amazon Technologies, Inc.',    
 'amcs-tachyon.com' : 'Amazon Technologies, Inc.',
 'fireoscaptiveportal.com' : 'Amazon Technologies, Inc.',
 'chtbl.com' : 'Chartable Holding Inc',
 'chrt.fm' : 'Chartable Holding Inc',                        
 'dillilabs.com' : 'Dilli Labs LLC',
 'megaphone.fm' :  'Spotify AB',
 'omny.fm' : 'Triton Digital, Inc.',
 'podtrac.com' : 'Podtrac Inc',
 'voiceapps.com' : 'Voice Apps LLC',
 'mittendorf.net' : 'individual',
 'doctorpooch.com' : 'Dilli Labs LLC',
 'kwimer.com' : 'Highwinds Network Group, Inc'}

In [12]:
for fqdn, details in dict_fqdns.items():
    if len(details['organization']) == 0:
        details['organization'] = manually_found_organs[details['domain_name']]

In [13]:
dict_fqdns

{'mlis.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'mlis',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'd3p8zr0ffa9t17.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd3p8zr0ffa9t17',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'api.amazonalexa.com': {'domain_name': 'amazonalexa.com',
  'SLD': 'amazonalexa',
  'subdomain': 'api',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'device-metrics-us-2.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'device-metrics-us-2',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'd29r7idq0wxsiz.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd29r7idq0wxsiz',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'd37b6sgey2y8gt.cloudfront.net': {'domain_

# Find ATSes

In [14]:
filter_list_base = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/filter_lists/"
tracking = utilities.read_json(os.path.join(filter_list_base,"tracking", "tracking_list.json"))
advertising = utilities.read_json(os.path.join(filter_list_base, "advertising", "advertising_list.json"))

In [15]:
for fqdn, details in dict_fqdns.items():
    if fqdn in tracking or fqdn in advertising:
        details['ATSes'] = "Advertising/Tracking"
        print(fqdn)
        continue
    if fqdn in tracking or fqdn in tracking:
        details['ATSes'] = "Advertising/Tracking"
        print(fqdn)
        continue
    else:
        details['ATSes'] = "Function"
        continue

device-metrics-us-2.amazon.com
fireoscaptiveportal.com
chtbl.com


### Mannualy detect advertiser

In [16]:
adtrack = ['podtrac', 'tritondigital', 'libsyn', 'streamtheworld', 'megaphone', 'omny']

In [17]:
for fqdn, details in dict_fqdns.items():
    if details["SLD"] in adtrack:
        details['ATSes'] = "Advertising/Tracking"

In [18]:
dict_fqdns

{'mlis.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'mlis',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'Function',
  'Party': ''},
 'd3p8zr0ffa9t17.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd3p8zr0ffa9t17',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'Function',
  'Party': ''},
 'api.amazonalexa.com': {'domain_name': 'amazonalexa.com',
  'SLD': 'amazonalexa',
  'subdomain': 'api',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'Function',
  'Party': ''},
 'device-metrics-us-2.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'device-metrics-us-2',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'Advertising/Tracking',
  'Party': ''},
 'd29r7idq0wxsiz.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd29r7idq0wxsiz',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'Function',
  'Part

# Find Parties

In [19]:
for fqdn, details in dict_fqdns.items():
    try:
        if details['organization'] == 'Amazon Technologies, Inc.':
            details['Party'] = "Platform-based"
            continue
        if details['SLD'] == 'garmincdn' or  details['SLD'] == 'doctorpooch' or details['SLD'] == 'youversionapi':
            details['Party'] = "First-Party"
            continue
        else:
            details['Party'] = "Third-Party"
    except:
        print(fqdn)

In [20]:
utilities.write_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/dict_fqdns_0307.json",dict_fqdns)

# Edit Final DataFrame

In [21]:
main_df = {'cat': [], 'skill_name': [], 'FQDN': [], 'domain_name': [], 'SLD': [], 'subdomain' : [] ,'organization': [], 'ATSes': [], 'Party': []}
for category in categories_echo:
    category_name = category.split('/')[-1]
    jsons = utilities.get_files_in_a_directory(os.path.join(category, "jsons"))
    for skill in jsons:
        ip_list = set()
        skill_name = skill.split('/')[-1].split('.')[0]
        skill_network_json = utilities.read_json(skill)
        for i in range(len(skill_network_json)):
            if 'ip' in skill_network_json[i]["_source"]["layers"].keys():
                ip_dst = skill_network_json[i]["_source"]["layers"]['ip']['ip.dst']
                if not ip_dst.startswith("10.42.") and not ip_dst.startswith("255.255.") and not ip_dst.startswith("8.8.") and not ip_dst.startswith("0.0.") and not ip_dst.startswith("224.0") and not ip_dst.startswith("239.255") and not ip_dst.startswith("192.168."):
                    if skill_name == 'B01J0DFHBE' and ip_dst == "142.202.65.6":
                        fqdn = skill_network_json[i]["_source"]["layers"]['ip']['ip.dst_host']
                    else:
                        fqdn = dns[ip_dst]
                        main_df['cat'].append(category_name)
                        main_df['skill_name'].append(skill_name)

                        main_df['FQDN'].append(fqdn)
                        main_df['domain_name'].append(dict_fqdns[fqdn]['domain_name'])
                        main_df['subdomain'].append(dict_fqdns[fqdn]['subdomain'])         
                        main_df['SLD'].append(dict_fqdns[fqdn]['SLD'])
                        main_df['organization'].append(dict_fqdns[fqdn]['organization'])
                        main_df['ATSes'].append(dict_fqdns[fqdn]['ATSes'])
                        main_df['Party'].append(dict_fqdns[fqdn]['Party'])

In [27]:
df = pd.DataFrame.from_dict(main_df)

In [28]:
#df[df['skill_name','cat']].drop_duplicates().count()
df[['cat','skill_name']].drop_duplicates().groupby('cat').count()

,skill_name
cat,
ConnectedCar,49
Dating,50
Fashion-Style,49
Health-Fitness,49
Navigation-TripPlanners,49
Pets-Animals,50
Religion-Spirituality,50
SmartHome,50
Wine-Beverages,50


In [24]:
df[['cat','skill_name']]

,cat,skill_name
0,Fashion-Style,B083PY1PGZ
1,Fashion-Style,B083PY1PGZ
2,Fashion-Style,B083PY1PGZ
3,Fashion-Style,B083PY1PGZ
4,Fashion-Style,B083PY1PGZ
...,...,...
966360,Health-Fitness,B07PJWHYXM
966361,Health-Fitness,B07PJWHYXM
966362,Health-Fitness,B07PJWHYXM
966363,Health-Fitness,B07PJWHYXM


In [25]:
df

,cat,skill_name,FQDN,domain_name,SLD,subdomain,organization,ATSes,Party
0,Fashion-Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
1,Fashion-Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
2,Fashion-Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
3,Fashion-Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
4,Fashion-Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
...,...,...,...,...,...,...,...,...,...
966360,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
966361,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
966362,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
966363,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based


## Edit Names

In [26]:
df = pd.read_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report_0307.csv")
df[['Skill Categories','skill_name']].drop_duplicates().groupby('Skill Categories').count()

,skill_name
Skill Categories,
Beverages,50
Connected Car,50
Dating,50
Fashion Style,50
Health,50
Navigation and TripPlanners,50
Pets,50
Smart Home,50
Sprituality,50


In [29]:
for index, row in df.iterrows():
    if row['cat'] == 'ConnectedCar':
        row['cat'] = 'Connected Car'
        
    if row['cat'] == 'Fashion-Style':
        row['cat'] = 'Fashion Style'
        
    if row['cat'] == 'SmartHome':
        row['cat'] = 'Smart Home'
        
    if row['cat'] == 'Wine-Beverages':
        row['cat'] = 'Beverages'
        
    if row['cat'] == 'Navigation-TripPlanners':
        row['cat'] = 'Navigation and TripPlanners'
        
    if row['cat'] == 'Health-Fitness':
        row['cat'] = 'Health'
        
    if row['cat'] == 'Pets-Animals':
        row['cat'] = 'Pets'
        
    if row['cat'] == 'Religion-Spirituality':
        row['cat'] = 'Sprituality'
    
df.rename(columns = {'cat' : 'Skill Categories', 'domain_name' : 'Domain Name', 'organization': 'Organization', 'ATSes':'Purpose'}, inplace = True)      

In [30]:
df

,Skill Categories,skill_name,FQDN,Domain Name,SLD,subdomain,Organization,Purpose,Party
0,Fashion Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
1,Fashion Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
2,Fashion Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
3,Fashion Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
4,Fashion Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
...,...,...,...,...,...,...,...,...,...
966360,Health,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
966361,Health,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
966362,Health,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
966363,Health,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based


# Save dataframe in csv format

In [31]:
df.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report_0307.csv", sep=',')

In [32]:
unique_rows = df.drop_duplicates()
unique_rows

,Skill Categories,skill_name,FQDN,Domain Name,SLD,subdomain,Organization,Purpose,Party
0,Fashion Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
136,Fashion Style,B083PY1PGZ,avs-alexa-14-na.amazon.com,amazon.com,amazon,avs-alexa-14-na,"Amazon Technologies, Inc.",Function,Platform-based
438,Fashion Style,B083PY1PGZ,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
2357,Fashion Style,B083PY1PGZ,prod.amcs-tachyon.com,amcs-tachyon.com,amcs-tachyon,prod,"Amazon Technologies, Inc.",Function,Platform-based
3757,Fashion Style,B083PY1PGZ,d3p8zr0ffa9t17.cloudfront.net,cloudfront.net,cloudfront,d3p8zr0ffa9t17,"Amazon Technologies, Inc.",Function,Platform-based
...,...,...,...,...,...,...,...,...,...
964263,Health,B07QN179C5,avs-alexa-14-na.amazon.com,amazon.com,amazon,avs-alexa-14-na,"Amazon Technologies, Inc.",Function,Platform-based
964605,Health,B07QN179C5,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
964654,Health,B07PJWHYXM,prod.amcs-tachyon.com,amcs-tachyon.com,amcs-tachyon,prod,"Amazon Technologies, Inc.",Function,Platform-based
964656,Health,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based


In [33]:
unique_rows.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report_unique_0307.csv", sep=',')

In [34]:
len(unique_rows)

1807

In [3]:
import pandas as pd
df = pd.read_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report_unique_0307.csv")

In [7]:
df

,Unnamed: 0,Skill Categories,skill_name,FQDN,Domain Name,SLD,subdomain,Organization,Purpose,Party
0,0,Fashion Style,B083PY1PGZ,device-metrics-us-2.amazon.com,amazon.com,amazon,device-metrics-us-2,"Amazon Technologies, Inc.",Advertising/Tracking,Platform-based
1,136,Fashion Style,B083PY1PGZ,avs-alexa-14-na.amazon.com,amazon.com,amazon,avs-alexa-14-na,"Amazon Technologies, Inc.",Function,Platform-based
2,438,Fashion Style,B083PY1PGZ,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
3,2357,Fashion Style,B083PY1PGZ,prod.amcs-tachyon.com,amcs-tachyon.com,amcs-tachyon,prod,"Amazon Technologies, Inc.",Function,Platform-based
4,3757,Fashion Style,B083PY1PGZ,d3p8zr0ffa9t17.cloudfront.net,cloudfront.net,cloudfront,d3p8zr0ffa9t17,"Amazon Technologies, Inc.",Function,Platform-based
...,...,...,...,...,...,...,...,...,...,...
1802,964263,Health,B07QN179C5,avs-alexa-14-na.amazon.com,amazon.com,amazon,avs-alexa-14-na,"Amazon Technologies, Inc.",Function,Platform-based
1803,964605,Health,B07QN179C5,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based
1804,964654,Health,B07PJWHYXM,prod.amcs-tachyon.com,amcs-tachyon.com,amcs-tachyon,prod,"Amazon Technologies, Inc.",Function,Platform-based
1805,964656,Health,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",Function,Platform-based


In [17]:
df_3rd = df[df['Purpose'] == 'Advertising/Tracking']
df_3rd.groupby('Skill Categories')

,Unnamed: 0,skill_name,FQDN,Domain Name,SLD,subdomain,Organization,Purpose,Party
Skill Categories,,,,,,,,,
Beverages,17,17,17,17,17,15,17,17,17
Connected Car,25,25,25,25,25,21,25,25,25
Dating,14,14,14,14,14,11,14,14,14
Fashion Style,25,25,25,25,25,22,25,25,25
Health,16,16,16,16,16,15,16,16,16
Navigation and TripPlanners,25,25,25,25,25,23,25,25,25
Pets,16,16,16,16,16,14,16,16,16
Smart Home,18,18,18,18,18,15,18,18,18
Sprituality,14,14,14,14,14,11,14,14,14


In [11]:
df_3rd

,Unnamed: 0,Skill Categories,skill_name,FQDN,Domain Name,SLD,subdomain,Organization,Purpose,Party
20,6375,Fashion Style,B07CB3ZN6N,play.podtrac.com,podtrac.com,podtrac,play,Podtrac Inc,Advertising/Tracking,Third-Party
21,6415,Fashion Style,B07CB3ZN6N,edge1.pod.npr.org,npr.org,npr,edge1.pod,"National Public Radio, Inc.",Function,Third-Party
22,6437,Fashion Style,B07CB3ZN6N,edge2.pod.npr.org,npr.org,npr,edge2.pod,"National Public Radio, Inc.",Function,Third-Party
23,6825,Fashion Style,B07CB3ZN6N,traffic.megaphone.fm,megaphone.fm,megaphone,traffic,Spotify AB,Advertising/Tracking,Third-Party
24,6836,Fashion Style,B07CB3ZN6N,dcs.megaphone.fm,megaphone.fm,megaphone,dcs,Spotify AB,Advertising/Tracking,Third-Party
133,58846,Fashion Style,B072N6BNB1,traffic.megaphone.fm,megaphone.fm,megaphone,traffic,Spotify AB,Advertising/Tracking,Third-Party
134,58847,Fashion Style,B072N6BNB1,play.podtrac.com,podtrac.com,podtrac,play,Podtrac Inc,Advertising/Tracking,Third-Party
135,58863,Fashion Style,B072N6BNB1,edge1.pod.npr.org,npr.org,npr,edge1.pod,"National Public Radio, Inc.",Function,Third-Party
136,58875,Fashion Style,B072N6BNB1,edge2.pod.npr.org,npr.org,npr,edge2.pod,"National Public Radio, Inc.",Function,Third-Party
139,59970,Fashion Style,B072N6BNB1,chtbl.com,chtbl.com,chtbl,NaN,Chartable Holding Inc,Advertising/Tracking,Third-Party


In [ ]:
df_3rd

In [ ]:
df_first = unique_rows[unique_rows['Party'] == 'First-Party']
df_first['FQDN'].drop_duplicates()

### Thir party Analysis

In [ ]:
import pandas as pd
df = pd.read_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report.csv")
df_3rd = df[df['Party'] == 'non_Platform-based']
del df_3rd['Unnamed: 0']
df_3rd.groupby('cat').count()

In [ ]:
df_3rd

In [ ]:
df_3rd_tritondigital = df_3rd[df_3rd['SLD'] == 'tritondigital']
df_3rd_tritondigital.groupby('cat').size()
df_3rd_tritondigital

In [ ]:
df_3rd_tritondigital = df_3rd[df_3rd['SLD'] == 'podtrac']
df_3rd_tritondigital.groupby('Skill Categories').size()
df_3rd_tritondigital

In [ ]:
subgroup_skills = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/skills_data/subgrouped_skills.json")
df_3rd_tritondigital['skill_link'] = ""
for index, row in df_3rd_tritondigital.iterrows():
    df_3rd_tritondigital.loc[index, 'skill_link'] = subgroup_skills[row['cat']][row['skill_name']]['Skill_link'] 
df_3rd_tritondigital.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Rahmadi/reporttritondigital_.csv", sep=',')

In [ ]:
df_3rd_podtrac = df_3rd[df_3rd['SLD'] == 'podtrac']
df_3rd_podtrac.groupby('cat').size()
df_3rd_podtrac

df_3rd_podtrac['skill_link'] = ""
for index, row in df_3rd_podtrac.iterrows():
    df_3rd_podtrac.loc[index, 'skill_link'] = subgroup_skills[row['cat']][row['skill_name']]['Skill_link'] 
df_3rd_podtrac.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Rahmadi/report-podtrac.csv", sep=',')

In [ ]:
for index, row in df_3rd_podtrac.iterrows():
    file_name = row['skill_name'] + ".PCAPS"
    categoty = row['cat']
    